# Creating a Simple Agent with Tracing

In [1]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [2]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

Create a simple Nutrition Assistant Agent

In [4]:
nutrition_agent = Agent(
    name='Nutrition Assistant',
    instructions="""
    You are a helpful assistant giving out nutrition advice.
    You give concise answers.
    """
)

Let's execute the Agent:

In [5]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    Bananas are a healthy, convenient fruit. Benefits:
    - Nutrients: potassium (good for blood pressure), vitamin B6, vitamin C, fiber.
    - Low in fat and protein; provide quick energy from natural sugars.
    
    Nutritional snapshot (medium banana):
    - ~105 calories
    - ~3 g fiber
    - ~27 g carbs (about 14 g sugars)
    - ~400–450 mg potassium
    
    Tips:
    - Great as a snack or in smoothies, yogurt, or oatmeal.
    - If watching sugar intake (e.g., diabetes), portion size matters; pair with protein/fat to slow absorption.
    - Choose ripe bananas for sweetness; greener ones have more resistant starch.
    
    Overall: healthy as part of a balanced diet, with attention to portion if you’re watching sugars.
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


Streaming the answer to the screen, token by token

In [5]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

Overall, yes—bananas are healthy when eaten as part of a balanced diet.

Key points:
- Good nutrients: potassium, vitamin B6, vitamin C, dietary fiber.
- Moderate calories: ~105 kcal per medium banana.
- Carbs: mainly natural sugars and starch; provides quick energy plus some resistant starch when less ripe.
- Benefits: supports heart health (potassium), digestion (fiber and resistant starch), and energy.

Considerations:
- If you’re watching carbs or blood sugar, portion size matters; ripe bananas have more sugar.
- Can be paired with protein/fat to improve satiety and glycemic response.

Bottom line: a healthy, convenient fruit to include regularly, unless you need strict carb control.

_Good Job!_